In [1]:
import math
import pandas as pd

# Reference 

1. Association of Structural Engineers of the Philippines 2015 Edition, National Structural Code of the Philippines Volume 1, Quezon City, Philippines.
2. American Concrete Institute, ACI-440R-17

In [2]:
input = {
    "project_name" : "Makati Life Nurture Medical Center <i>Formerly Ospital ng Makati District 1</i>",
    "project_location" : "Malugay Street, Barangay Bel Air, Makati City ",
    "designer" : "Engr. Albert Pamonag, M.Eng",
    "l" : 7.32, # m
    "w" : 305, # mm
    "d" : 546, # mm
    "h" : 609.6, # mm
    "fc" : 34.5, # N/mm2
    "fy" : 414, # N/mm2
    "Ms" : 214, # Service Load Moment in kN-m
    "Mn" : 361, # phiMn without FRP in kN-m
    "Mu" : 399, # factored moment based on loading in kN-m
    "db" : 28.6, # mm
    "Es" : 200, # GPa
    "exposure" : {
        "condition" : "interior",
        "fiber" : "carbon"
    },
    "frp" : {
        "tf" : 1.02, # thickness per ply in mm
        "ffu" : 621, # ultimate tensile strength in N/mm2
        "efu" : 0.015, # rupture strain in mm/mm
        "Ef" : 37000, # modulus of elasticity of FRP laminates in N/mm2
        "psi_f" : 0.85
    }
}

In [3]:
def eRF(exposure,type):
    data = {
        "interior" : {
            "carbon" : 0.95,
            "glass" : 0.75,
            "aramid" : 0.85
        },
        "exterior" : {
            "carbon" : 0.85,
            "glass" : 0.65,
            "aramid" : 0.75
        },
        "aggressive" : {
            "carbon" : 0.85,
            "glass" : 0.50,
            "aramid" : 0.70
            },
    }
    results = data[exposure][type]
    return results

# Flexural Strengthening

## Design material properties :
Reference Two - Section 9.4

### Design Ultimate Tensile Strength of FRP:
<center>$ f_{fu} = C_{E} f^{*}_{fu} $</center>

In [4]:
ffu = eRF(input['exposure']['condition'],input['exposure']['fiber'])*input['frp']['ffu']
print('Value : ' + str(round(ffu,3)) + " N/mm^2")

Value : 589.95 N/mm^2


### Design Rupture Strain of FRP reinforcement attained at failure:
<center>$ \varepsilon_{fu} = C_{E} f^{*}_{fu} $</center>

In [5]:
efu = eRF(input['exposure']['condition'],input['exposure']['fiber'])*input['frp']['efu']
print('Value : ' + str(round(efu,3)) + " N/mm^2")

Value : 0.014 N/mm^2


### Properties of the existing reinforcing steel and Properties of the externally bonded FRP reinforcement:

In [6]:
def beta_1(fc): 
    ##NSCP 2015 422.2.2.4.3
    value = 1.05 - 0.05*(fc/1000)
    if(17 <= fc and fc <= 28):
        value = 0.85
    elif (28 <= fc and fc < 55):
        value = 0.85 - ((0.05*(fc-28))/7)
    else:
        value = 0.65
    return value
    
def areaCircle(db):
    pi = math.pi
    value = (pi/4)*(math.pow(db,2))
    return value
    
beta1 = beta_1(input['fc'])
ec = 4700*math.sqrt(input['fc'])
n_bot = 3
n_frp = 2
area_steel = n_bot*areaCircle(input['db'])
af = n_frp*input["frp"]['tf']*input['w']

print(area_steel)
print(af)

1927.2728451977305
622.2


$ \beta _{1} $

In [7]:
print(beta1)

0.8035714285714286


### Modulus of Elasticity of Concrete
<center> $ E_{c} = 4700 \sqrt{ f_{c} } $ </center>

In [8]:
print(ec)

27606.249292506214


## Properties of the concrete:

### Determine the existing state of strain on the soffit
<center>$ \epsilon_{bi} = \frac{ M _{DL} \times \left( d_{f} - kd \right) }{ I_{cr} \times E_{c} } $</center>

In [48]:
import math
n_value = input["Es"]*1000 / (4700*math.sqrt(input["fc"]))
n = max(6, math.ceil(n_value))
check_Icr = input["w"] * math.pow(input["d"]-input["h"]/2,3)/3 + n * area_steel * math.pow((input["d"] - input["h"]/2),2)
print (n, check_Icr/1e6)

8 2323.617586200962


In [9]:
MDL = 97.63 # kN-mm
df = input['h']
k = 0.334
d = input['d']
Icr = 2.471e6 ##mm4 TODO double check this one
ec_convert = ec/1000 # kN / mm2

ebi = ( MDL * (df - (k*d)) )  / ( Icr * ec_convert ) 
print(ebi)

0.0006114641207566778


## Determine the design strain of the FRP system
<center> $ \epsilon _{fd} = 0.41 \times \sqrt{ \frac{ f_{c}^{'} }{ 2 \times E_{f} \times t_{f} } } $ </center>

In [10]:
efd_init = input['fc']/(2*input["frp"]['tf']*input["frp"]['Ef'])
efd = 0.41*math.sqrt(efd_init)
print(efd)

0.008765515428730905


## Compare Design Strain (efd) to Rupture Strain (efu)

In [12]:
multiplier = 0.9 #TODO verify multiplier
limit = multiplier * efu
if efd <= limit:
    result = "Debonding controls the design of the FRP system"
else:
    result = "FRP rupture becomes the governing failure mode"

print(result)

Debonding controls the design of the FRP system


## Computation of the compression block

In [13]:
def cSolver():## //Numerical Solver
        
    ##--> Data for the tabulations
    epsilon_fe_arr = []
    espilon_c_arr = []
    eespilon_s_arr = []
    e_c_prime_arr = []
    beta_one_arr = []
    alpha_one_arr = []
    c_init_arr = []

    c_init = 0.20*input['d'] ##initialize depth
    i = 0
    
    while i <= 20:
       
        epsilon_fe = 0.003*( (input['h'] - c_init) /c_init) - ebi
        espilon_c = (0.009 + ebi)*(c_init/(input['h'] - c_init))
        espilon_s = (0.009 + ebi)*((input['d'] - c_init)/(input['h'] - c_init))

        fs = input['Es']*espilon_s
        fs = 414 ##TODO fix on the yields
            
        f_fe = input['frp']['Ef']*0.009
            
        e_c_prime = 1.7*input['fc']/ec
        beta_one = ((4*e_c_prime )- espilon_c)/((6*e_c_prime)-(2*espilon_c))
        alpha_one = ((3*e_c_prime*espilon_c) - math.pow(espilon_c,2))/(3*beta_one*math.pow(e_c_prime,2))    
        c_init = (area_steel*fs + af*f_fe)/(alpha_one*input['fc']*beta_one*input['w'])

        epsilon_fe_arr.append(epsilon_fe)
        espilon_c_arr.append(espilon_c)
        eespilon_s_arr.append(espilon_s)
        e_c_prime_arr.append(e_c_prime)
        beta_one_arr.append(beta_one)
        alpha_one_arr.append(alpha_one)
        c_init_arr.append(c_init)
    
        c = c_init
        i += 1
    results = {
        "value" : c,
        "epsilon_fe" : epsilon_fe_arr,
        "espilon_c" : espilon_c_arr,
        "espilon_s" : eespilon_s_arr,
        "e_c_prime" : e_c_prime_arr,
        "beta_one" : beta_one_arr,
        "alpha_one" : alpha_one_arr,
        "c_init" : c_init_arr,
    }
        
    return results

c_data = cSolver()
df = pd.DataFrame({
      'c (mm)': c_data['c_init'],
      'ε_fe': c_data['epsilon_fe'],    
      'ε_c': c_data['espilon_c'],
      'ε_s': c_data['espilon_s'],
      'ε_c\'': c_data['e_c_prime'],
      'β1': c_data['beta_one'],
      'α': c_data['alpha_one'],
      })
df

,c (mm),ε_fe,ε_c,ε_s,ε_c',β1,α
0,144.206181,0.013136,0.002097,0.008390,0.002125,0.748418,0.885024
1,127.904714,0.009070,0.002978,0.008298,0.002125,0.812856,0.918720
2,132.616347,0.010687,0.002552,0.008342,0.002125,0.777975,0.925807
3,130.765101,0.010179,0.002672,0.008330,0.002125,0.786999,0.928148
4,131.444495,0.010374,0.002625,0.008335,0.002125,0.783363,0.927637
5,131.187991,0.010302,0.002642,0.008333,0.002125,0.784683,0.927886
6,131.283854,0.010329,0.002636,0.008334,0.002125,0.784183,0.927801
7,131.247888,0.010319,0.002638,0.008333,0.002125,0.784370,0.927834
8,131.261362,0.010322,0.002637,0.008334,0.002125,0.784299,0.927821
9,131.256312,0.010321,0.002637,0.008334,0.002125,0.784326,0.927826


### Computation of Flexural Strength

#### Steel Contribution to Bending
<center> $ M_{ns} = A_{s} f_{s} \left( d - \frac{ \beta_{1} c }{2} \right) $ </center>

In [14]:
## Computation of Flexural Strength
beta_one_final = c_data["beta_one"][-1]
c_final = c_data["value"]

## Computation of Steel Contribution to Bending
fs = 414  # Assumed yield stress for steel, to be reviewed
As = area_steel
Af = af

# Compute nominal moment strength (Mns)
Mns = As * fs * (input['d'] - ((beta_one_final * c_final) / 2))

# Compute FRP contribution
f_fe = input['frp']['Ef'] * 0.009

print (Mns/1e6)

394.5778889129859


#### FRP Contribution to Bending
<center> $ M_{nf} = A_{s} f_{fe} \left( d - \frac{ \beta_{1} c }{2} \right) $ </center>

In [15]:
## Computation of Steel Contribution to Bending
fs = 414  # Assumed yield stress for steel, to be reviewed
As = area_steel
Af = af

# Compute nominal moment strength (Mns)
Mnf = Af * f_fe * (input["h"] - ((beta_one_final * c_final) / 2))

# Compute FRP contribution
f_fe = input['frp']['Ef'] * 0.009

print (Mnf/1e6)

115.63959396224989


### Computation Design Flexural Strength of the Section
<center>$ \phi M_{n} = \phi \left[ M_{ns} + \psi_{f} M_{nf} \right] $</center>

In [16]:
# Strength reduction factors
psif = 0.85  # For FRP contribution (verify with design codes)
phib = 0.90  # For flexural strength reduction

# Compute factored moment capacity
phiMn = phib * (Mns + (psif * Mnf))

# Convert to MN·m
phiMn_MNm = phiMn / 1e6

print(phiMn_MNm)

443.5843894028084


In [17]:
if phiMn_MNm >= input["Mu"]:
    result = "pass"
else:
    result = "fail"

print (result)

pass


### Calculate service stresses in reinforcing steel and the FRP

<center> $ k = \sqrt{ \left( \rho_{s} \frac{E_{s}}{E_{c}} + \rho_{f} \frac{E_{f}}{E_{c}} \right)^{2} + 2.0 \times \left[ \rho_{s} \frac{E_{s}}{E_{c}} + \rho_{f} \frac{E_{f}}{E_{c}} \times \left( \frac{d_{f}}{d} \right) \right] } - \left( \rho_{s} \frac{E_{s}}{E_{c}} + \rho_{f} \frac{E_{f}}{E_{c}} \right) $ </center>

In [ ]:
def rho(As,b,d):
    value = As/(b*d)
    return value

rho_s = rho(area_steel,input["d"],input["w"])
rho_f = rho(af,input["d"],input["w"])

Es = 200 # GPa
Ec = ec/1000 # Mpa -> KPa ??????
Ef = input['frp']['Ef']/1000

k1 = math.pow(rho_s*(Es/Ec) + rho_f*(Ef/Ec),2)
k2 = 2*(rho_s*(Es/Ec) + rho_f*(Ef/Ec)*(input['h']/input['d']))
k3 = rho_s*(Es/Ec) + rho_f*(Ef/Ec)

k = math.sqrt(k1 + k2) - k3
kd = k*input["d"]
print(k, kd)

0.3433113171777111 187.44797917903026


### Serviceability Check 
Reference Two : Section 10.2.8



<center> $ f_{s,s} = \frac{ \left[ M_{s} + \varepsilon_{bi} A_{f} E_{f} \left( d_{f} - \frac{kd}{3} \right) \right] \left( d_{f} - \frac{kd}{3} \right) E_{s} }{ A_{s} E_{s} \left( d_{f} - \frac{kd}{3} \right) \left( d_{f} - \frac{kd}{3} \right) + A_{f} E_{f} \left( d_{f} - \frac{kd}{3} \right) \left( d_{f} - \frac{kd}{3} \right) } $ </center><br>

The Stress in the steel reinforcement under service load should be limited to 80 percent of the yield strength. <br> 

<center> $ f_{s,s} \leq 0.80 f_{y} $ </center>


In [34]:
nume = (input["Ms"]*1000 + (ebi * Af * Ef) * (input["h"] - ((k*d) / 3))) * (d - ((k*d))) * Es
deno = As * Es * (d - ((k*d)/ 3)) * (d - ((k*d))) + Af * Ef * (input["h"] - ((k*d) / 3)) * (input["h"] - ((k*d)))
fss1 = (nume / deno) * 1000

limit = 0.8 * input["fy"]

if fss1 > limit:
    fss = limit
else:
    fss = fss1

print(limit)
print(fss)

331.20000000000005
220.37547507060782


### Check creep rupture limit at service of the FRP
<center> $ f_{f,s} = f_{s,s} \left( \frac{E_{f}}{E_{s}} \right) \left( \frac{ d_{f} - kd }{ d - kd } \right)  - \varepsilon_{bi} E_{f} $ </center>

In [ ]:
# fss = 278
df = input["h"]
kn_to_n = 1000 
kn_to_m = 1/1000
depth_ratio = (df - kd) / (d - kd)
part1 = fss/1000 * (Ef / Es) * depth_ratio
eps_term = ebi * Ef

ffs = (part1 - eps_term)*kn_to_n 
limit = 0.55 * ffu

if ffs <= limit:
    result = "The stress level in the FRP is within the recommended sustained plus cyclic stress limit."
else:
    result = "The stress level in the FRP is NOT within the recommended sustained plus cyclic stress limit."

print (limit)
print (ffs)
print (result)

324.47249999999997
37.928487034829246
The stress level in the FRP is within the recommended sustained plus cyclic stress limit.
